In [7]:
import json
import requests
import pickle

In [8]:
all_post_lists = []

In [9]:
for site in ["buzzfolder.com", "examequip.com", "gradepack.com", "quizlookup.com", "wikicram.com"]:
    page = 1
    print(site)
    while True:
        if(page%100==0):
            print(page)
        response = requests.get(url="https://" + site + "/wp-json/wp/v2/posts", params={"page":page, "per_page": 100})
        # the "page > 200" part limits it to more recent questions, which are more likely to be "hot"
        # remove it to get all questions (this may take a _long_ time)
        # TODO: prevent fetching posts we already have
        if(response.status_code!=200 or page > 200):
            break
        all_post_lists.append(response.json())
        page+=1
    pickle.dump(all_post_lists, open("v2/"+site+".raw_posts", 'wb'))
    print("data written")

buzzfolder.com
100
200
data written
examequip.com
100
200
data written
gradepack.com
100
200
data written
quizlookup.com
100
200
data written
wikicram.com
100
200
data written


In [10]:
# multi site mode
all_post_lists = []
for site in ["buzzfolder.com", "examequip.com", "gradepack.com", "quizlookup.com", "wikicram.com"]:
    this_site_file = open("v2/"+site+".raw_posts", 'rb')
    site_lists = pickle.load(this_site_file)
    this_site_file.close()
    all_post_lists.append(site_lists)
print(len(all_post_lists))

5


In [25]:
all_post_lists[0][0][0]['questions'][0]

[{'blog-post-questions-question': 'Injuncti&ocy;ns &acy;re frequently used by c&ocy;urts t&ocy; force employees to complete their contr&acy;ctu&acy;l obligations with their employers.',
  'blog-post-questions-answer': '',
  'blog-post-questions-anchor-link': 'injunctions-are-frequently-used-by-courts-to-force-employees-to-complete-their-contractual-obligations-with-their-employers'},
 {'blog-post-questions-question': 'When we discussed h&ocy;w the c&ocy;ncerns &acy;ss&ocy;ci&acy;ted with infertility &acy;nd control are played out in the lives of infertile couples, which of the following mattered in terms specific to the expression of SELF?',
  'blog-post-questions-answer': '',
  'blog-post-questions-anchor-link': 'when-we-discussed-how-the-concerns-associated-with-infertility-and-control-are-played-out-in-the-lives-of-infertile-couples-which-of-the-following-mattered-in-terms-specific-to-the-expression-of-self'},
 {'blog-post-questions-question': 'The first step in relieving the p&acy;

In [11]:
all_questions = set()
for site in all_post_lists:
    for post_list in site:
        for post in post_list:
            for question in post['questions'][0]:
    #             print(question['blog-post-questions-question'])
                all_questions.add(question['blog-post-questions-question'])
                if(question['blog-post-questions-answer']!=''):
                    print(question['blog-post-questions-answer'])

In [12]:
len(all_questions)

242887

In [13]:
import unidecode
import html

In [14]:
all_questions_formatted = set()
for q in all_questions:
    all_questions_formatted.add(unidecode.unidecode(html.unescape(q)).strip())

In [15]:
len(all_questions_formatted)

241089

In [16]:
# search for a string
for q in all_questions_formatted:
    if "honorlock" in q.lower():
        print(q)

The following references are provided for you to reference during the exam. Open the following links within Honorlock (they should open in a new tab), and use Ctrl+1, Ctrl+2, and Ctrl+3 to navigate between them. Alternatively, you can scroll back to here and reference these documents as needed. If you can't access these or if clicking on the link does not open a new tab, exit out of the exam immediately and contact Jeremy. You will need this information to complete the exam. Periodic Table of the Elements Exam 1 Reference Sheet
When your exam is over (at 4pm for most of you), do the following:  Scan the scratch pages that you used (using Camscanner or equivalent), to create a single PDF file. Be sure to scan ALL of the pages that you wrote on. Pages not scanned will not be graded! Open the file on your phone to be sure that it has what you want. Place your scan in the chosen cloud storage folder. From your computer, upload your scan as requested from your cloud storage folder to your C

In [18]:
questions_out = open('questions_out.txt', 'w')
for q in all_questions_formatted:
    questions_out.write(q + '\n')
questions_out.close()

In [ ]:
# split into multiple files for the question browser
!split -l 20 questions.txt questions/q_ --numeric-suffixes --additional-suffix=.txt --suffix-length=5